In [85]:
import agentpy as ap
import matplotlib.pyplot as plt
import numpy as np

# Visualization
import seaborn as sns
import IPython

In [86]:
#Funciones Auxiliares
def iteratorlist(node):

  out = []

  for i in(node):
    out.append(i)

  return out

In [87]:
class Node(ap.Agent):

    #Constructor
    def setup(self):

        #Reglas de desplazamiento
        self.desp = [[0, 1], [0, -1], [1, 0], [-1, 0]]

        #Reglas de desplazamiento
        self.xlim = 20
        self.ylim = 20

        #Crear ID
        self.id = None

        #Ruta de camino
        self.parents = []

        #Tipo de rama
        self.branch = 0

    #Asignar ID
    def assign_id(self, id):

        #Asignar ID
        self.id = int(id)
        self.parents.append(int(id))

    #Reglas de desplazamiento
    def setup_rules(self, xlim, ylim):

        #Reglas de desplazamiento
        self.xlim = xlim
        self.ylim = ylim

    #Obtener posicion
    def get_position(self, world):

        #Get position
        position = world.positions[self]

        return position

    #Obtener estado siguiente
    def get_next_state(self, world):

        #Obtener posicion
        position = self.get_position(world)

        #Obtener valores de alrededores
        values = []
        for i in range(len(self.desp)):

          #Obtener posicion tras desplazamiento
          pos_x = position[0] + self.desp[i][0]
          pos_y = position[1] + self.desp[i][1]

          #Obtener condicion
          if((pos_x < 0) or (pos_y < 0) or (pos_x > self.xlim - 1) or (pos_y > self.ylim - 1)):
            values.append(-1)
          else:
            #Obtener posicion siguiente
            value = world.agents[(pos_x, pos_y)].condition
            for i in value:
              values.append(i)
              break

        return values

    #Propagar trayectoria
    def propagate(self, world):

      # Get node position
      position = self.get_position(world)

      # Get sorrounding
      sorroundings = self.get_next_state(world)

      # Propagate path
      for i in range(len(sorroundings)):
          
          # Neighbour position
          pos = position[0] + self.desp[i][0], position[1] + self.desp[i][1]
         
          # Free space
          if(sorroundings[i] == 0):

            # Change condition to active node (2)
            world.agents[pos].condition = 2

            # Heritage return path
            world.agents[pos].parents += self.parents

            # Objective node
          if (sorroundings[i] == 4):

            # Change objective node condition
            world.agents[pos].condition = 5

            # Heritage return path
            world.agents[pos].parents += self.parents
            break   



    #Propagar trayectoria
    def propagate_bidirectional(self, world):
      
      # Get node position
      position = self.get_position(world)

      # Get sorrounding
      sorroundings = self.get_next_state(world)

      # Propagate path
      for i in range(len(sorroundings)):
          
          # Neighbour position
          pos = position[0] + self.desp[i][0], position[1] + self.desp[i][1]
         
          # Free space
          if(sorroundings[i] == 0):

            # Change condition to active node (2)
            world.agents[pos].condition = 2
            world.agents[pos].branch = self.branch

            # Heritage return path
            world.agents[pos].parents += self.parents

            # Objective node
          if (sorroundings[i] == 2):
            
            current_branch = iteratorlist(world.agents[pos].branch)
            
            # Verify if the branch is from other tree
            if(current_branch[0] != self.branch):

              # Change objective node condition
              world.agents[pos].condition = 5

              # Heritage return path
              world.agents[pos].parents += self.parents
              break



In [88]:
class RandomMap(ap.Model):

    #Metodo de configuracion
    def setup(self):

      self.mode = self.p["mode"]

      # Create agents floor 
      n_agents = int(self.p["dim"]**2)
      self.floor_agents = ap.AgentList(self, n_agents, Node)

      #Create space
      self.floor = ap.Grid(self, [self.p["dim"]] * 2, track_empty=True)
      self.floor.add_agents(self.floor_agents)

      # Define free space
      self.floor_agents.condition = 0

      # Define obstacles
      obstacle_index = np.random.randint(len(self.floor_agents), size = int(self.p["obs_den"]*n_agents))
      for i in range(obstacle_index.shape[0]):
         self.floor_agents[obstacle_index[i]].condition = 1

      # Create root node
      self.start_index = np.random.randint(len(self.floor_agents))
      self.floor_agents[self.start_index].condition = 2

      # Create goal node
      self.goal_index = np.random.randint(len(self.floor_agents))
      self.floor_agents[self.goal_index].condition = 4

      # Assign ID
      for i in range(len(self.floor_agents)):
        self.floor_agents[i].assign_id(i)

      # Assign branches
      self.floor_agents[self.start_index].branch = 0
      self.floor_agents[self.goal_index].branch = 1

    #Iteracion
    def step(self):
       
      # Find active nodes
      active_nodes = []
    
      for agent in self.floor_agents:
        
          if (agent.condition == 2):
            active_nodes.append(agent)
          if (self.mode == 1):
            if (agent.condition == 4):
              active_nodes.append(agent)
          
          # Verify condition
          if(agent.condition == 2 or agent.condition == 4):
            active_nodes.append(agent)

      # Execute find algorithm
      # A) Find by unidirectional width
      if(self.mode == 0):
      
        # Propagate active nodes
        for agent in active_nodes:

          # Grow path
          agent.propagate(self.floor)

      # B) Find by bidirectional width
      if(self.mode == 1):

        # Propagate active nodes
        for agent in active_nodes:

          # Grow path
          agent.propagate_bidirectional(self.floor)


      for agent in self.floor_agents:

        # Verify 5 condition
        if (agent.condition == 5):

          # Recover return path
          self.draw_path(agent)
          
          # Stop execution
          self.stop()
        




    #Marcar camino
    def draw_path(self, node):

      #Obtener camino del objetivo
      path = []
      for parent in node.parents:
        path.append(parent)

      #Cambiar estado de los nodos
      for j in range(len(path)):
        for agent in self.floor_agents:
          if(agent.id == path[j]):
            agent.condition = 3
    

    #Finalizar simulacion
    def end(self):

        print("Camino encontrado!")
        self.floor_agents[self.goal_index].condition == 3

In [89]:
# Inicializar parametros
parameters = {
    'obs_den': 0.3, # Percentage of grid covered by trees
    'dim': 20, # Height and length of the grid
    'mode': 1,  #Modo: 0: Anchura, 1: Bidireccional
}

In [90]:
#Crear instancia
model = RandomMap(parameters)

In [91]:
# Create single-run animation with custom colors
def rgb2hex(r, g, b):

  var = '#%02x%02x%02x' % (r, g, b)

  return var

# Create single-run animation with custom colors
def animation_plot(model, ax):
    attr_grid = model.floor.attr_grid('condition')
    color_dict = {0:rgb2hex(255, 255, 255), 1:rgb2hex(0, 0, 0), 2:rgb2hex(128, 128, 128), 3:rgb2hex(0, 255, 0), 4:rgb2hex(255, 0, 0), 5:rgb2hex(0, 0, 255), None:rgb2hex(255, 255, 255)}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)

fig, ax = plt.subplots()
animation = ap.animate(model, fig, ax, animation_plot, steps = 25)
IPython.display.HTML(animation.to_jshtml(fps=15))